<a href="https://colab.research.google.com/github/JMML2021/0401_Registre-de-logs/blob/main/0401_Registre-de-logs%20Rev%20C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# importing libraries
from urllib.request import urlretrieve
#import py7zr
import pandas as pd
import os
import re
from datetime import datetime
import pytz
#import geocoder
import folium
import time
import urllib.request
import json
import geopandas
import matplotlib.pyplot as plt
from datetime import datetime

In [8]:
# Activo Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Abro archivo

df = list(open('/content/drive/MyDrive/Web_access_log-akumenius.com.txt'))

In [10]:
# 0.- Abro por la mitad la linea

def custom_split(sepr_list, str_to_split):
    separatorsIzq = "[", "]", "\s", ","

    # create regular expression dynamically
    regular_exp = '|'.join(map(re.escape, sepr_list))
    return re.split(regular_exp, str_to_split)


In [11]:
# 1.- Separo la primera parte
def ladoIzquierdo(texto1):
  x = re.split("\s", texto1)
  direccion = x[0]
  print('Lado Izquierdo ', ladoIzquierdo(pp[0]))
  IP = x[1]
  return (direccion, IP)


#



In [13]:
#2.- Separo la parte central.

def custom_split(str_to_split):
    separatorsCentro = "[", "]", "\s", ",",' '
    # create regular expression dynamically
    regular_exp = '|'.join(map(re.escape, separatorsCentro))

    return re.split(regular_exp, str_to_split)

#pp1= custom_split(pp[1])
#print('Lado Derecho: ', pp1)    


In [55]:
#3.- Separamos lado Derecho.

def custom_splitDcha( texto3):
    separatorsDcha = '"', ';', ' '
    # create regular expression dynamically
    regular_exp = '|'.join(map(re.escape, separatorsDcha))
    return re.split(regular_exp, texto3)



In [57]:
data = {'Direccion': [],
    'IPs': [],
    'Fecha': [],
    'Direccion': [],
    'Navegador': []}
df_salida = pd.DataFrame(data)

for s, textoLinea  in enumerate(df):
    #f = open(destFile, 'a+') 

    if s>=1 and s <=501:
        #print(s,' TODO ', textoLinea)
        pp= custom_split(textoLinea)  
        #for p,q in enumerate(pp):
          #print(p,q)
        #print(s,len(pp), ' linea ', pp[0], pp[1] )
        #pp[0]
        nueva_fila = {'Direccion': pp[0],
          'IPs': pp[1],
          'Fecha': pp[5],
          'Direccion': pp[9],
          'Navegador': pp[14]+pp[15]+pp[16]}
      #print(s, nueva_fila)
      #Añadiendo una fila al dataframe
        df_salida = df_salida.append(nueva_fila, ignore_index=True) 


df_salida

,Direccion,IPs,Fecha,Navegador
0,*,127.0.0.1,23/Feb/2014:03:10:31,"""Apache(internaldummy"
1,*,127.0.0.1,23/Feb/2014:03:10:31,"""Apache(internaldummy"
2,*,127.0.0.1,23/Feb/2014:03:10:31,"""Apache(internaldummy"
3,*,127.0.0.1,23/Feb/2014:03:10:31,"""Apache(internaldummy"
4,*,127.0.0.1,23/Feb/2014:03:10:31,"""Apache(internaldummy"
...,...,...,...,...
496,/destinos-baratos/destinosEstrelles/hoteles-en...,66.249.76.216,23/Feb/2014:03:22:22,"""Mozilla/5.0(compatible;Googlebot/2.1;"
497,/hoteles-baratos/ofertas-hotel-Dorint-Am-Nurbu...,66.249.76.216,23/Feb/2014:03:22:23,"""Mozilla/5.0(compatible;Googlebot/2.1;"
498,/hoteles-baratos/ofertas-hotel-Blue-Jack-Tar-e...,66.249.76.216,23/Feb/2014:03:22:25,"""Mozilla/5.0(compatible;Googlebot/2.1;"
499,/hoteles-baratos/ofertas-hotel-Campanile-Le-Ma...,66.249.76.216,23/Feb/2014:03:22:28,"""Mozilla/5.0(compatible;Googlebot/2.1;"


In [ ]:
ip = geocoder.ip("80.28.221.123")
print(ip.city)
print(ip)
print(ip.latlng)

Terrassa
<[OK] Ipinfo - Geocode [Terrassa, Catalonia, ES]>
[41.5667, 2.0167]


In [ ]:
location = ip.latlng

map = folium.Map(location=location, zoom_start=105)
folium.CircleMarker(location=location, radius=15, color="blue").add_to(map)
folium.Marker(location).add_to(map)
map

In [ ]:
https://cheatography.com/davechild/cheat-sheets/regular-expressions/

https://cheatography.com/davechild/cheat-sheets/regular-expressions/

In [ ]:
folium.Map(location=[45.5236, -122.6750],
           tiles='Mapbox',
           API_key='your.API.key')
map

In [ ]:
https://datasciencesphere.com/project/track-location-ip-address-python-geocoder/

https://datasciencesphere.com/project/track-location-ip-address-python-geocoder/

In [ ]:
ip = geocoder.ip("me")
ip

print(ip.latlng)

<[OK] Ipinfo - Geocode [North Charleston, South Carolina, US]>

In [ ]:
import requests

url = 'https://maps.googleapis.com/maps/api/geocode/json'
r = requests.get(url, params={
    'sensor': 'false', 
    'address': 'Centre Pompidou, Paris, FR'
})

results = r.json()['results']
results
#location = results[0]['geometry']['location']
#lat, lng = location['lat'], location['lng']

[]

https://janakiev.com/blog/geocoding-in-python/

In [ ]:
location = geocoder.ip("66.249.76.216")
print(ip.city)
print(ip)
print(ip.latlng)

location = ip.latlng
lat, lng = location[0], location[1]

#location = results[0]['geometry']['location']
#lat, lng = location['lat'], location['lng']

print(lat, lng, location)

Brussels
<[OK] Ipinfo - Geocode [Brussels, Brussels Capital, BE]>
[50.8505, 4.3488]
50.8505 4.3488 [50.8505, 4.3488]
